In [ ]:
import pandas as pd

# Load the data from the uploaded file
file_path = r'E:\Root\Chmoinformatics\표면휘발식\2023 ODE\B9A28580.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet2')

# Display the first few rows of the data
data.head()

In [ ]:
# Load all sheets in the Excel file
data = pd.read_excel(file_path, sheet_name=None)

#use for loop to print all sheet data
for sheet in data:
    print(sheet)
    print(data[sheet].head())
    print()

In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN

# Create a presentation object
prs = Presentation()

# Add a slide with a title and content layout
slide_layout = prs.slide_layouts[6]  

Rs_list = [760/4, 570/4, 1250/4, 880/4]

Vs_list = []

Fair_list = []

ρdv_list = []

ρdv_re_list = []

# 특정 column만 불러오기
# BKC back만 
for i, sheet in enumerate(data):
    data_bkc_back = data[sheet][['time(hr)', 'BKC amount (ug/m3)']] # BKC amount -back (raw)

    # 결측치 제거
    data_bkc_back = data_bkc_back.dropna()

    import numpy as np
    from scipy.optimize import curve_fit

    # Extracting the relevant data
    x_data = data_bkc_back['time(hr)'].values
    y_data = data_bkc_back['BKC amount (ug/m3)'].values

    # Define the piecewise function
    def piecewise_func_back(x, a, b, xr):
        return np.where(x <= xr, 
                        (a/b) * (1 - np.exp(-b * x)), 
                        (a/b) * ((1 - np.exp(-b * xr)) * np.exp(-b * (x - xr))))

    # Initialize best fit parameters and the corresponding error
    best_params = None
    lowest_error = float('inf')

    # Iterate over a range of possible xr values
    for xr in np.linspace(4/60, 5/60, 100):  # Fine-grained search within the interval
        try:
            # Fit the model to the data
            params, _ = curve_fit(lambda x, a, b: piecewise_func_back(x, a, b, xr), x_data, y_data, p0=[1, 1])

            # Calculate the sum of squared errors
            fitted_values = piecewise_func_back(x_data, params[0], params[1], xr)
            error = np.sum((y_data - fitted_values) ** 2)

            # Update best parameters if this fit is better
            if error < lowest_error:
                best_params = (params[0], params[1], xr)
                lowest_error = error
        except RuntimeError:
            # In case the fitting fails for a specific set of parameters
            continue

    # Best parameters and the corresponding error
    print(best_params, lowest_error)
    # Unpack the updated best parameters
    a_best, b_best, xr_best = best_params

    # Generate fitted values with updated parameters
    fitted_y = piecewise_func_back(x_data, a_best, b_best, xr_best)

    import matplotlib.pyplot as plt
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    # Calculate RMSE, MAE, and R²
    rmse_back = np.sqrt(mean_squared_error(y_data, fitted_y))
    mae_back = mean_absolute_error(y_data, fitted_y)
    r_squared = r2_score(y_data, fitted_y)

    # Sum of Squared Errors (SSE) is already calculated as lowest_error_updated
    sse = lowest_error

    # 맨 마지막 column name 불러오기
    print(data[sheet].columns[-1])

    # 맨 마지막 column name을 :로 split
    split_name_back = data[sheet].columns[-1].split(':')
    print(split_name_back[-1]) 

    # Plotting with updated fit
    plt.figure(figsize=(15, 9))
    plt.scatter(x_data, y_data/1000, label='Data', color='blue')
    plt.plot(x_data, fitted_y/1000, label='Fitted Model', color='red', linestyle='--')
    xticks_labels = [f'{hour:.2f} ({hour*60}min)' for hour in x_data]
    plt.xticks(x_data, xticks_labels, rotation=45)
    plt.axvline(x=xr_best, color='green', linestyle=':', label=f'$x_r = {xr_best:.4f}$ (Approx. {xr_best*60:.2f} minutes)')

    plt.title('Instantaneous BKC concentration over time') # 샘플 이름으로 변경
    plt.xlabel('time(hr)')
    plt.ylabel('Instantaneous BKC concentration (μg/L)')
    plt.legend()

    # Split the fitted equation string into multiple lines
    fitted_equation_lines = [
        f"Fitted Equation:",
        f"y = a/b(1-e^(-bx)) for x <= xr",
        f"y = a/b(1-e^(-bxr))e^(-b(x-xr)) for x > xr",
        f"where a = {a_best:.2f}, b = {b_best:.2f}"
    ]

    figure_of_merit = [
        f"Figure_of_merit:",
        f"Coefficient of Determination (R²) = {r_squared:.2f}",
        f"Sum of Squared Errors (SSE) = {sse:.2f}"   
    ]

    N = 0.66
    h = 1.5

    Vs = (b_best - N) * h # 빈 리스트 만들고 append
    Vs_list.append(Vs)

    ρdv = (Vs*18*0.0181)/(3600*9.8)
    ρdv_list.append(ρdv)

    ρdv_re = ρdv * 10 ** 4
    ρdv_re_list.append(ρdv_re)

    V = 105
    wf = 500 / 1000000
    Rs = (Rs_list[i]) * (60*1000)

    Fair = (a_best * V) / (Rs * wf) # 빈 리스트 만들고 append
    Fair_list.append(Fair)

    value_back = [
        f"Value of Vs and Fair:",
        f"Vs = {Vs:.2f} m/hr",
        f"Fair = {Fair:.4f}",
        f"ρdv^2 = {ρdv_re:.4f}*10^(-4) g/m^3 * m^2"   
    ]

    

    #Position the text in the upper right corner, below the legend
    # plt.text(0.63, 0.68, '\n'.join(fitted_equation_lines_back), fontsize=9,
    #         transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))
    # plt.text(0.63, 0.55, '\n'.join(figure_of_merit_back), fontsize=9,
    #         transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))
    # plt.text(0.63, 0.42, '\n'.join(value_back), fontsize=9,
    #         transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))
    plt.grid(True)
    #파일명을 split_name_back으로 변경
    plt.savefig(r"E:\Root\Chmoinformatics\표면휘발식\2023 ODE" + "/" + "{}_240124.png".format(str(split_name_back[-1])), dpi=300) 
    plt.show()
    
    slide_back = prs.slides.add_slide(slide_layout)

    fitted_equation_lines_txt = [
        "Fitted Equation:",
        "y = a/b(1-e^(-bx)) for x <= xr",
        "y = a/b(1-e^(-bxr))e^(-b(x-xr)) for x > xr",
        f"where a = {a_best:.2f}, b = {b_best:.2f}",
        "",
        "Figure of merit:",
        f"Coefficient of Determination (R²) = {r_squared:.2f}",
        f"Sum of Squared Errors (SSE) = {sse:.2f}",
        "",
        "Value of Vs, Fair and ρdv^2 :",
        f"Vs = {Vs:.2f} m/hr",
        f"Fair = {Fair:.4f}",
        f"ρdv^2 = {ρdv_re:.4f}*10^(-4) g/m^3 * m^2"
    ]

    fitted_equation_text = "\n".join(fitted_equation_lines_txt)

    # Add a text box for the fitted equation text
    left, top, width, height = Inches(0.5), Inches(1), Inches(4), Inches(3)
    txBox = slide_back.shapes.add_textbox(left, top, width, height)
    tf = txBox.text_frame
    tf.text = fitted_equation_lines_txt[0]  # Set the first line of text

    # Add the remaining lines
    for line in fitted_equation_lines_txt[1:]:
        p = tf.add_paragraph()
        p.text = line
        p.font.size = Pt(12)
        p.alignment = PP_ALIGN.LEFT
        
        if line.startswith("Figure"):
            p.font.size = Pt(18)
        elif line.startswith("Value"):
            p.font.size = Pt(18)

    # Insert the plot image
    img_path_back = r"E:\Root\Chmoinformatics\표면휘발식\2023 ODE" + "/" + "{}_240124.png".format(str(split_name_back[-1]))
    left_img, top_img, width_img, height_img = Inches(3.5), Inches(0.8), Inches(5), Inches(4)
    pic = slide_back.shapes.add_picture(img_path_back, left_img, top_img, height=height)

Vs_avg = sum(Vs_list) / len(Vs_list)
Fair_avg = sum(Fair_list) / len(Fair_list)
ρdv_avg = sum(ρdv_list) / len(ρdv_list)
ρdv_re_avg = sum(ρdv_re_list) / len(ρdv_re_list)

# Add a slide and insert the average value of Vs and Fair
slide_layout_avg = prs.slide_layouts[6]
slide_avg = prs.slides.add_slide(slide_layout_avg)

fitted_equation_lines_txt_avg = [
    "Average value of Vs and Fair:",
    f"Vs = {Vs_avg:.2f} m/hr",
    f"Fair = {Fair_avg:.4f}",
    f"ρdv^2 = {ρdv_re_avg:.4f}*10^(-4) g/m^3 * m^2"
]

fitted_equation_text_avg = "\n".join(fitted_equation_lines_txt_avg)

# Add a text box for the fitted equation text

left, top, width, height = Inches(0.5), Inches(1), Inches(4), Inches(3)
txBox = slide_avg.shapes.add_textbox(left, top, width, height)
tf = txBox.text_frame
tf.text = fitted_equation_lines_txt_avg[0]  # Set the first line of text

# Add the remaining lines
for line in fitted_equation_lines_txt_avg[1:]:
    p = tf.add_paragraph()
    p.text = line
    p.font.size = Pt(12)
    p.alignment = PP_ALIGN.LEFT

prs.save(r"E:\Root\Chmoinformatics\표면휘발식\2023 ODE\bkc_ODE_240124.pptx")

In [ ]:
#list들을 dataframe으로 만들기
df_Vs = pd.DataFrame(Vs_list)
df_Fair = pd.DataFrame(Fair_list)
df_ρdv = pd.DataFrame(ρdv_list)
df_Rs = pd.DataFrame(Rs_list)

#dataframe 합치기
df_values = pd.concat([df_Vs, df_Fair, df_ρdv, df_Rs], axis=1)
df_values.columns = ['Vs', 'Fair', 'ρdv', 'Rs']
df_values.index = ['BKC front auto', 'BKC back auto', 'BKC front manual', 'BKC back manual']
df_values.to_excel(r'E:\Root\Chmoinformatics\표면휘발식\2023 ODE\bkc_ODE_values_240124.xlsx')

In [ ]:
df = pd.read_excel(r'E:\Root\Chmoinformatics\표면휘발식\2023 ODE\B7EAF480.xlsx', sheet_name='Sheet3')
df.head()

In [ ]:
df_front_auto = df[['time (hr)', 'Front_auto_Normalization (ug)']].dropna()
df_front_auto.head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Grouping the data by 'Time (hr)' and calculating the mean and standard deviation
front_auto_grouped_data = df_front_auto.groupby('time (hr)').agg(
    ['mean', 'std']).reset_index()
front_auto_grouped_data.to_csv(r'E:\Root\Chmoinformatics\표면휘발식\2023 ODE\front_auto_grouped_data.csv')

In [ ]:
front_auto_time = front_auto_grouped_data['time (hr)']
front_auto_mean = front_auto_grouped_data['Front_auto_Normalization (ug)']['mean']
front_auto_std = front_auto_grouped_data['Front_auto_Normalization (ug)']['std']
front_auto_grouped_data

In [ ]:
#fitting front_auto_time and front_auto_mean using sigmoid function
from scipy.optimize import curve_fit
def sigmoid(x, L ,x0, k):
    y = L / (1 + np.exp(-k*(x-x0)))
    return (y)

p0 = [1,1,1] # this is an mandatory initial guess

popt, pcov = curve_fit(sigmoid, front_auto_time, front_auto_mean, p0, method='dogbox')

print(popt, pcov)

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(front_auto_time, sigmoid(front_auto_time, *popt), label='Fitted Curve', color='red', linestyle='--')
plt.errorbar(front_auto_time, front_auto_mean, yerr=front_auto_std, 
             fmt='o', ecolor='gray', capsize=5, linestyle='', color = 'black', marker='o', markersize=5)

plt.title('Accumulated BKC amount over time')
xticks_labels = [f'{hour:.2f} ({hour*60}min)' for hour in front_auto_time]
plt.xticks(front_auto_time, xticks_labels, rotation=45)
plt.xlabel('Time (hr)')
plt.ylabel('Accumulated BKC amount (μm)')
plt.text(0.5, 0.4, f'y = {popt[0]:.2f} / (1 + exp(-{popt[2]:.2f} * (x - {popt[1]:.2f})))',
            transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))
plt.grid(True)

# Calculating the area of the error bars
error_bar_area = np.sum(front_auto_std * 2)  # Multiply by 2 for both sides of the mean

plt.show(), error_bar_area

In [ ]:
df_back_auto = df[['time (hr)', 'Back_auto_Normalization (ug)']].dropna()
df_back_auto.head()

In [ ]:
back_auto_grouped_data = df_back_auto.groupby('time (hr)').agg(
    ['mean', 'std']).reset_index()
back_auto_grouped_data.to_csv(r'E:\Root\Chmoinformatics\표면휘발식\2023 ODE\back_auto_grouped_data.csv')

In [ ]:
index_to_drop = back_auto_grouped_data.index[-2]

# Drop the second-to-last row
back_auto_grouped_data_edit = back_auto_grouped_data.drop([9, 13, 14])

back_auto_grouped_data_edit

In [ ]:
# Extracting the time and mean values
back_auto_time = back_auto_grouped_data_edit['time (hr)']
back_auto_mean = back_auto_grouped_data_edit['Back_auto_Normalization (ug)']['mean']
back_auto_std = back_auto_grouped_data_edit['Back_auto_Normalization (ug)']['std']

import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

def sigmoid(x, L ,x0, k):
    y = L / (1 + np.exp(-k*(x-x0)))
    return (y)

p0 = [1,1,1] # this is an mandatory initial guess

popt, pcov = curve_fit(sigmoid, back_auto_time, back_auto_mean, p0, method='dogbox')

print(popt, pcov)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(back_auto_time, sigmoid(back_auto_time, *popt), label='Fitted Curve', color='red', linestyle='--')
plt.errorbar(back_auto_time, back_auto_mean, yerr=back_auto_std, 
             fmt='o', ecolor='gray', capsize=5, linestyle='', color = 'black', marker='o', markersize=5)

plt.title('Accumulated BKC amount over time')
xticks_labels = [f'{hour:.2f} ({hour*60}min)' for hour in back_auto_time]
plt.xticks(back_auto_time, xticks_labels, rotation=45)
plt.xlabel('Time (hr)')
plt.ylabel('Accumulated BKC amount (μm)')
plt.text(0.03, 0.7, f'y = {popt[0]:.2f} / (1 + exp(-{popt[2]:.2f} * (x - {popt[1]:.2f})))',
            transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))
plt.grid(True)

# Calculating the area of the error bars
error_bar_area = np.sum(back_auto_std * 2)  # Multiply by 2 for both sides of the mean

plt.show(), error_bar_area

In [ ]:
df_back_auto_except = df[['time (hr)', 'Back_auto_Normalization_except0731 (ug)']].dropna()
df_back_auto_except.head()

In [ ]:
back_auto_except_grouped_data = df_back_auto_except.groupby('time (hr)').agg(
    ['mean', 'std']).reset_index()
back_auto_except_grouped_data.to_csv(r'E:\Root\Chmoinformatics\표면휘발식\2023 ODE\back_auto_except_grouped_data.csv')

In [ ]:
# Extracting the time and mean values
back_auto_except_time = back_auto_except_grouped_data['time (hr)']
back_auto_except_mean = back_auto_except_grouped_data['Back_auto_Normalization_except0731 (ug)']['mean']
back_auto_except_std = back_auto_except_grouped_data['Back_auto_Normalization_except0731 (ug)']['std']

def sigmoid(x, L ,x0, k):
    y = L / (1 + np.exp(-k*(x-x0)))
    return (y)

p0 = [1,1,1] # this is an mandatory initial guess

popt, pcov = curve_fit(sigmoid, back_auto_except_time, back_auto_except_mean, p0, method='dogbox')

print(popt, pcov)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(back_auto_except_time, sigmoid(back_auto_except_time, *popt), label='Fitted Curve', color='red', linestyle='--')
plt.errorbar(back_auto_except_time, back_auto_except_mean, yerr=back_auto_except_std, 
             fmt='o', ecolor='gray', capsize=5, linestyle='', color = 'black', marker='o', markersize=5)

plt.title('Average Normalized Values over Time(Back_auto_except0731)')
plt.xlabel('Time (hr)')
plt.ylabel('Average Normalized (ug)')
plt.text(0.01, 0.7, f'y = {popt[0]:.2f} / (1 + exp(-{popt[2]:.2f} * (x - {popt[1]:.2f})))',
            transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))
plt.grid(True)

# Calculating the area of the error bars
error_bar_area = np.sum(back_auto_except_std * 2)  # Multiply by 2 for both sides of the mean

plt.show(), error_bar_area

In [ ]:
df_front_manual = df[['time (hr)', 'Front_manual_Normalization (ug)']].dropna()
df_front_manual.head()

In [ ]:
# Grouping the data by 'Time (hr)' and calculating the mean and standard deviation
front_manual_grouped_data = df_front_manual.groupby('time (hr)').agg(
    ['mean', 'std']).reset_index()
front_manual_grouped_data.to_csv(r'E:\Root\Chmoinformatics\표면휘발식\2023 ODE\front_manual_grouped_data.csv')

In [ ]:
# Extracting the time and mean values
front_manual_time = front_manual_grouped_data['time (hr)']
front_manual_mean = front_manual_grouped_data['Front_manual_Normalization (ug)']['mean']
front_manual_std = front_manual_grouped_data['Front_manual_Normalization (ug)']['std']

def sigmoid(x, L ,x0, k):
    y = L / (1 + np.exp(-k*(x-x0)))
    return (y)

p0 = [1,1,1] # this is an mandatory initial guess

popt, pcov = curve_fit(sigmoid, front_manual_time, front_manual_mean, p0, method='dogbox')

print(popt, pcov)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(front_manual_time, sigmoid(front_manual_time, *popt), label='Fitted Curve', color='red', linestyle='--')
plt.errorbar(front_manual_time, front_manual_mean, yerr=front_manual_std, 
             fmt='o', ecolor='gray', capsize=5, linestyle='', color = 'black', marker='o', markersize=5)

plt.title('Accumulated BKC amount over time')
xticks_labels = [f'{hour:.2f} ({hour*60}min)' for hour in front_manual_time]
plt.xticks(front_manual_time, xticks_labels, rotation=45)
plt.xlabel('Time (hr)')
plt.ylabel('Accumulated BKC amount (μg)')
plt.text(0.05, 0.7, f'y = {popt[0]:.2f} / (1 + exp(-{popt[2]:.2f} * (x - {popt[1]:.2f})))',
            transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))
plt.grid(True)
plt.grid(True)

# Calculating the area of the error bars
error_bar_area = np.sum(front_manual_std * 2)  # Multiply by 2 for both sides of the mean

plt.show(), error_bar_area

In [ ]:
df_back_manual = df[['time (hr)', 'Back_manual_Normalization (ug)']].dropna()
df_back_manual.head()

In [ ]:
# Grouping the data by 'Time (hr)' and calculating the mean and standard deviation
back_manual_grouped_data = df_back_manual.groupby('time (hr)').agg(
    ['mean', 'std']).reset_index()
back_manual_grouped_data.to_csv(r'E:\Root\Chmoinformatics\표면휘발식\2023 ODE\back_manual_grouped_data.csv')

In [ ]:
# Extracting the time and mean values
back_manual_time = back_manual_grouped_data['time (hr)']
back_manual_mean = back_manual_grouped_data['Back_manual_Normalization (ug)']['mean']
back_manual_std = back_manual_grouped_data['Back_manual_Normalization (ug)']['std']

def sigmoid(x, L ,x0, k):
    y = L / (1 + np.exp(-k*(x-x0)))
    return (y)

p0 = [1,1,1] # this is an mandatory initial guess

popt, pcov = curve_fit(sigmoid, back_manual_time, back_manual_mean, p0, method='dogbox')

print(popt, pcov)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(back_manual_time, sigmoid(back_manual_time, *popt), label='Fitted Curve', color='red', linestyle='--')
plt.errorbar(back_manual_time, back_manual_mean, yerr=back_manual_std, 
             fmt='o', ecolor='gray', capsize=5, linestyle='', color = 'black', marker='o', markersize=5)

plt.title('Accumulated BKC amount over time')
xticks_labels = [f'{hour:.2f} ({hour*60}min)' for hour in back_manual_time]
plt.xticks(back_manual_time, xticks_labels, rotation=45)
plt.xlabel('Time (hr)')
plt.ylabel('Accumulated BKC amount (μg)')
plt.text(0.05, 0.7, f'y = {popt[0]:.2f} / (1 + exp(-{popt[2]:.2f} * (x - {popt[1]:.2f})))',
            transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))
plt.grid(True)

# Calculating the area of the error bars
error_bar_area = np.sum(back_manual_std * 2)  # Multiply by 2 for both sides of the mean

plt.show(), error_bar_area

In [ ]:
scatter_df = pd.read_excel(r'E:\Root\Chmoinformatics\표면휘발식\2023 ODE\231212data.xlsx', sheet_name='Sheet2')

scatter_df

In [ ]:
#draw scatter plot with scatter_df
import matplotlib.pyplot as plt
import numpy as np

time = scatter_df['분사시간(min)']/60

plt.figure(figsize=(10, 6))
plt.scatter(time, scatter_df['공간체적당 포집량(ug/m^3)']/1000)
plt.title('Instantaneous BKC concentration over time')
xticks_labels = [f'{hour:.2f} ({hour*60}min)' for hour in time]
plt.xticks(time, xticks_labels, rotation=45)
plt.xlabel('Time (hr)')
plt.ylabel('BKC amount (μg/L)')
plt.xlim(0, 14/60)
plt.ylim(0, 0.025)    
plt.grid(True)
plt.show()